# Become a movie director

Let's use Scrapy to get some information about the 250 top rated movies on <a href="http://www.imdb.com/" target="_blank">IMDB</a>.

1. Install `Scrapy`:

In [ ]:
#!pip install scrapy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


2. Create a folder where you will store the scripts and results of this exercise. Create a file named `imdb1.py` in that folder.

fait :src/imdb1.py


3. Start by writing code in the script to:
* import os, logging, scrapy, and CrawlerProcess from scrapy.crawler

ETAPE DE COMPREHENSION ET FAMILIARISATION AVEC SCRAPY

In [3]:
import os
import logging
import requests
from parsel import Selector



In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/132.0.0.0 Safari/537.36",
    "Accept-Language": "fr-FR,fr;q=0.9,en;q=0.8",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.google.com/",
}

url = "https://www.imdb.com/chart/top/"   # ou "/chart/top/" pour anglais

try:
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
except Exception as e:
    print("Erreur requête :", type(e).__name__, str(e))
    exit(1)

# On crée un objet Selector → c'est ce qui permet .css() et .xpath()
sel = Selector(text=resp.text)

print(f"Statut : {resp.status_code}  |  Taille : {len(resp.text):,} caractères\n")

# Extraction des films (sélecteurs actuels ~2025–2026)
films = sel.css('li.ipc-metadata-list-summary-item')

print(f"Nombre de films détectés : {len(films)}\n")

Statut : 200  |  Taille : 1,815,464 caractères

Nombre de films détectés : 25



In [5]:
premier_film=films[0] 
print(premier_film)

<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable" aria-disabled="false"></span><div class="sc-fc35a1ef-1 lmHCrT cli-parent li-compact"><div class="sc-fc35a1ef-0 hTMtRz"><div class="sc-d0224b4e-0 jfogmY cli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Morgan Freeman and Tim Robbins in Les Évadés (1994)" class="ipc-image" loading="lazy" src="https://m.media-amazon.com/images/M/MV5BOTc2MjMxNzIyMF5BMl5BanBnXkFtZTgwNDQ2MjEyMDE@._V1_QL75_UY133_CR5,0,90,133_.jpg" srcset="https://m.media

In [6]:
#premier titre
premier_titre = premier_film.css('h3.ipc-title__text::text').get()
print(premier_titre)

#premier lien
#on doit requeter une autre page pour les earnings
base_url="https://www.imdb.com"
film_relative_url = premier_film.css('a.ipc-title-link-wrapper::attr(href)').get('')
film_url = base_url+film_relative_url  # → https://www.imdb.com/title/tt0111161/
print(film_url)

#partie financiere sur une autre page
detail_resp = requests.get(film_url, headers=headers, timeout=10)
detail_sel = Selector(text=detail_resp.text)

#on cumule le filtrage avec la balise parente et les classes, puis on cible le bon item avec l'attribut data-testid
gross_worldwide = detail_sel.css(
    'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
    'span.ipc-metadata-list-item__list-content-item::text'
).get(default='?').strip()

#budget sur le meme mode
budget = detail_sel.css(
    'li[data-testid="title-boxoffice-budget"] '
    'span.ipc-metadata-list-item__list-content-item ::text'
).get()

print("Budget :", budget)
print("Gross mondial :", gross_worldwide)

# Le rating
premier_rating = premier_film.css("span.ipc-rating-star--rating::text").get().replace(",", ".")
premier_rating

Les Évadés
https://www.imdb.com/fr/title/tt0111161/?ref_=chttp_t_1
Budget : 25 000 000 $US (estimé)
Gross mondial : 29 334 033 $US


'9.3'

In [7]:
import re

def parse_vote_count(film):
    # Exemple de texte réel que tu obtiens souvent
    texte = "".join(film.css("span.ipc-rating-star--voteCount::text").getall())
    # → typiquement : ' (3,2\xa0M)'   ou ' (1,45\xa0M)'   ou ' (456\xa0K)'   ou ' (12345)'

    # Regex qui capture le nombre ET le suffixe optionnel (M/K)
    match = re.search(r'([\d,.]+)\s*([KM])?', texte, re.IGNORECASE)

    if match:
        nombre_str = match.group(1)          # '3,2' ou '1,45' ou '456'
        suffixe    = match.group(2) or ''    # 'M' ou 'K' ou ''

        # Nettoyage
        nombre_str = nombre_str.replace(',', '.').strip()   # '3.2'

        # Multiplicateur
        multi = 1
        if suffixe.upper() == 'M':
            multi = 1_000_000
        elif suffixe.upper() == 'K':
            multi = 1_000

        try:
            votes = int(float(nombre_str) * multi)

        except ValueError:
            print("Conversion impossible")
    else:
        print("Aucun nombre trouvé")            # → 320000
    return votes

In [8]:

parse_vote_count(premier_film)

3200000

In [10]:
import requests
from scrapy import Selector

url = "https://www.imdb.com/chart/top/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

r = requests.get(url, headers=headers)

response = Selector(text=r.text)

films = response.css("li[class='ipc-metadata-list-summary-item']")
classement = 0
dict_films = {}

for film in films:
    
    ### Création du dico d'un film
    dict_film = {}
    
    ### On crée le classement nous meme
    classement += 1
    dict_film['classement'] = classement
    
    ### On récupère le titre
    titre = film.css("h3[class='ipc-title__text']::text").get()
    dict_film['title'] = titre
    
    ### On récupère l'URL
    base_url="https://www.imdb.com"
    film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get()
    url_film = base_url+film_relative_url
    dict_film['url'] = url_film
    
    ### On récupère le rating
    rating = film.css("span.ipc-rating-star--rating::text").get()
    dict_film['rating'] = rating

    
    ### On récupère le nombre de votants
    nb_voteurs_raw = film.css("span.ipc-rating-star--voteCount::text").getall()
    nb_voteurs = parse_vote_count(film)
    dict_film['nb_voters'] = nb_voteurs

    ### on suit l'url relative 
    #---------------------------
    base_url="https://www.imdb.com"
    film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get()
    film_url = base_url+film_relative_url

    detail_resp = requests.get(film_url, headers=headers, timeout=10)
    detail_sel = Selector(text=detail_resp.text)    
    brut_mondial = detail_sel.css(
        'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
        'span.ipc-metadata-list-item__list-content-item::text'
    ).get(default='?').strip()
    dict_film['brut_mondial']  = brut_mondial
    #---------------------

    dict_films[f'film number {classement}'] = dict_film

dict_films

{'film number 1': {'classement': 1,
  'title': 'Les Évadés',
  'url': 'https://www.imdb.com/fr/title/tt0111161/?ref_=chttp_t_1',
  'rating': '9,3',
  'nb_voters': 3200000,
  'brut_mondial': '29\u202f334\u202f033\xa0$US'},
 'film number 2': {'classement': 2,
  'title': 'Le Parrain',
  'url': 'https://www.imdb.com/fr/title/tt0068646/?ref_=chttp_t_2',
  'rating': '9,2',
  'nb_voters': 2200000,
  'brut_mondial': '250\u202f933\u202f053\xa0$US'},
 'film number 3': {'classement': 3,
  'title': 'The Dark Knight : Le Chevalier noir',
  'url': 'https://www.imdb.com/fr/title/tt0468569/?ref_=chttp_t_3',
  'rating': '9,1',
  'nb_voters': 3100000,
  'brut_mondial': '1\u202f008\u202f477\u202f382\xa0$US'},
 'film number 4': {'classement': 4,
  'title': 'Le Parrain, 2ᵉ partie',
  'url': 'https://www.imdb.com/fr/title/tt0071562/?ref_=chttp_t_4',
  'rating': '9,0',
  'nb_voters': 1500000,
  'brut_mondial': '48\u202f204\u202f606\xa0$US'},
 'film number 5': {'classement': 5,
  'title': '12 Hommes en colère

FIN DE L'INTRODUCTION AVEC SCRAPY

4. Create a first spider called `imdb_spider` with:
* name `imdb`
* start_urls `https://www.imdb.com/chart/top/`

This spider should scrape the ranking, the title, the url, the total earnings, the rating and the number of voters for the first movie of the charts.

Set up the `CrawlerProcess`.
Save the results to a `imdb1.json` file.

VERSION LOCALE TEST NOTEBOOK

In [ ]:
#VERSION LOCALE TEST NOTEBOOK

import scrapy
import requests
import logging
import os
from scrapy.crawler import CrawlerProcess

class imdb_spider(scrapy.Spider):
    name = "imdb" 
    allowed_domains = ["imdb.com"] 
    
    start_urls = ["https://www.imdb.com/chart/top/"] 
    
    def parse(self, response): 
        films = response.css("li[class='ipc-metadata-list-summary-item']")
        
        for film in films:
            
            ### On crée le classement nous meme
            classement += 1
            dict_film['classement'] = classement
            
            ### On récupère le titre
            titre = film.css("h3[class='ipc-title__text']::text").get()
            
            ### On récupère l'URL
            base_url="https://www.imdb.com"
            film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get()
            url_details = base_url+film_relative_url
            
            ### On récupère le rating
            rating = film.css("span.ipc-rating-star--rating::text").get()

            ### On récupère le nombre de votants
            print(f"nb_voteurs_raw = {film}")
            nb_voteurs = parse_vote_count(film)

            ### on suit l'url relative pour scraper le revenubrut mondial
            #---------------------------
            detail_resp = requests.get(url_details, headers=headers, timeout=10)
            detail_sel = Selector(text=detail_resp.text)    
            revenu_brut_mondial = detail_sel.css(
                'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
                'span.ipc-metadata-list-item__list-content-item::text'
            ).get(default='?').strip()

            yield {
                'classement': classement, 
                'title': titre, 
                'url_details': url_details, 
                'revenu_brut_mondial': revenu_brut_mondial,
                'rating': rating, 
                'nb_voters': nb_voteurs
                }            
            
#on prepare le fichier de sortie
filename = "imdb1.json"
if os.path.exists(filename):
    os.remove(filename)

    #on prepare le crawling process
#si le fichier existe deja, on le supprime pour eviter la concatenation de plusieurs runs
if filename in os.listdir('src/'):
        os.remove('src/' + filename)

# Declaration du process de crawling avec des settings personnalisés
## USER_AGENT => simule un navigateur pour eviter les blocages basiques
## LOG_LEVEL => niveau de log pour n'avoir que les infos et pas les debug
## FEEDS => endroit et format de sortie des données scrapees
## plus info sur ces settings et  plus=> https://docs.scrapy.org/en/latest/topics/settings.html?highlight=settings#settings
process = CrawlerProcess(settings = {
    "USER_AGENT": ("Chrome/140.0.0.0"),
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        '01-Become_a_movie_director/' + filename : {"format": "json"},
    }
})
#on lance le crawling avec notre spider
process.crawl(imdb_spider) 
process.start()

***

# VERSION PROJET SCRAPY EXTERNE
## Guide pas-à-pas : Créer et lancer un projet Scrapy pour IMDb

## 1. Création automatique de l’arborescence du projet

Ouvrir un terminal (bash / cmd / PowerShell) et taper avec le nom de ton projet (ici "imdb_scraper") :

```bash
scrapy startproject imdb_scraper
```
La sortie doit etre similaire à ceci :
```
New Scrapy project 'imdb_scraper', using template directory 'C:\Users\pkoub\anaconda3\envs\jedha-scrapy\Lib\site-packages\scrapy\templates\project', created in:
    C:\Projects\formation\jedha\dsfs-ft-40\semaine2\scrapy\imdb_scraper

You can start your first spider with:
    cd imdb_scraper
    scrapy genspider example example.com
```
L'arborescence suivante a été générée par Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    ← Fichier de config global (Scrapy le lit depuis ici)
└── imdb_scraper/                 ← Module Python importable (le "vrai" package)
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               ← Les réglages du projet
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```

#### Pourquoi deux dossiers "imdb_scraper" l’un dans l’autre ?
-   Le dossier extérieur (imdb_scraper/ racine)
    -   C’est juste un conteneur pour tout le projet: il contient scrapy.cfg (le fichier qui dit à Scrapy "je suis un projet Scrapy")
    -   Tu lances les commandes (scrapy crawl, scrapy genspider, etc.) depuis ce dossier
    -   C’est le dossier que tu versionnes avec Git, que tu mets sur **GitHub**, que tu déploies, etc.
-   Le dossier intérieur (imdb_scraper/imdb_scraper/)
    -   C’est un vrai **module** Python (package)
    -   Il a un __init__.py → Python le reconnaît comme un package importable
    -   ***C’est depuis ce dossier que tu fais les imports dans tes spiders (crée le dossier 'utils' dans le 2e 'imdb_scraper/') :***
``` 
        from imdb_scraper.items import MonItem
        from imdb_scraper.utils.parsing import parse_vote_count
```
Se placer dans le dossier racine du projet (le 1er imdb_scraper/) pour lancer les commandes Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    
└── imdb_scraper/                 
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```

## 2. Création du spider imdb1 (ou le nom que tu veux)

Ouvrir un terminal (bash / cmd / PowerShell) et taper :

```bash
scrapy scrapy genspider imdb1 imdb.com
```
#### ceci génère un fichier 'imdb1.py'(le spider) dans le dossier **spiders**.
-   Modifier / compléter ce fichier qui contient une classe **Imdb1Spider** qui hérite de **scrapy.Spider**
-   Vérifier avec :
```bash
scrapy list
```
La sortie doit afficher le nom de ton spider :
```
imdb1
```
nb: si vous modifiez le nom de la classe du spider, vérifiez que le nom du spider (attribut name) correspond au nom de la classe en minuscules sans "Spider" (ex: class Imdb1Spider → name = "imdb1")

## 3. Modifier les fichiers **settings.py** et **imdb1.py**

    -   `USER_AGENT` = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    -   `FEED`
    -   ...
    --> On ne les met plus dans une classe !
## 4. Lancement du crawling et collecte des données

Ouvrir un terminal (bash / cmd / PowerShell) et taper :

```bash
scrapy crawl imdb1 -o output/imdb.json -L INFO
```

-   -o output/imdb.json : indique à Scrapy de sauvegarder les résultats dans un fichier JSON (dans le dossier output/)
-   -L INFO : définit le niveau de logging à INFO pour avoir des logs plus lisibles pendant le crawling"

## 5. check du résultat de la collecte -> le fichier de sortie
-   Ouvrir le fichier `output/imdb.json` pour vérifier que les données ont bien été collectées et sauvegardées au format JSON.

## La classe du projet scrapy externe (imdb_scraper/imdb_scraper/spiders/imdb1.py) :
Pour info: elle sera lancée pour recupérer des données depuis le ***dossier racine du projet*** (le 1er imdb_scraper/) avec la commande 
```bash
scrapy crawl imdb1 -o output/imdb.json -L INFO`
```
#### la version pour utilisation dans notebook est laissée en commentaire pour comparer les deux versions. les import sont gérés par Scrapy, ainsi que les logs, et beaucoup d'autres fonctionnalités (gestion des erreurs, des redirections, des requetes multiples, etc. --> infos dans le fichier settings.py)


In [ ]:
import scrapy
# import requests              # <-- supprimé : on n'utilise plus requests dans un spider Scrapy
# import logging
# import os
# from scrapy.crawler import CrawlerProcess          # <-- pas besoin ici (lancé via commande scrapy crawl)
# from parsel import Selector                        # <-- pas besoin (response est déjà un Selector)

# On suppose que parse_vote_count est dans utils/parsing.py
#pour externaliser le code de parsing et le rendre réutilisable dans d'autres spiders du projet
# créer un repertoire (ex utils/) et un fichier "parsing.py" pour y mettre le code "def votre_fonction(): ..."
#      l'importer avec 
#                   from imdb_scraper.utils.parsing import votre_fonction

# Version locale de parse_vote_count (recommandée pour commencer)
def parse_vote_count(vote_parts):
    """
    Extrait le nombre de votes à partir des parties extraites.
    Exemple : [' (', '3,2\xa0M', ')'] → 3200000
    """
    if len(vote_parts) < 3:
        return 0
    
    raw = vote_parts[1].replace('\xa0', '').strip()  # '3,2M'
    
    multi = 1
    if 'M' in raw.upper():
        multi = 1_000_000
        raw = raw.rstrip('Mm')
    elif 'K' in raw.upper():
        multi = 1_000
        raw = raw.rstrip('Kk')
    
    raw = raw.replace(',', '.').strip()
    
    try:
        return int(float(raw) * multi)
    except (ValueError, TypeError):
        return 0


class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    
    start_urls = ["https://www.imdb.com/chart/top/"]
    
    def parse(self, response):
        classement = 0     
        
        for film in response.css("li.ipc-metadata-list-summary-item"):
            classement += 1
            
            # ### On récupère le titre
            titre_raw = film.css("h3.ipc-title__text::text").get()
            titre = titre_raw.split('. ', 1)[-1].strip() if titre_raw else '?'  # ← nettoyé
            
            # ### On récupère l'URL
            # base_url="https://www.imdb.com"
            film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get('')
            if not film_relative_url:
                continue
            
            # url_details = response.urljoin(film_relative_url)   # ← on le fait dans Request
            
            # ### On récupère le rating
            # rating = film.css("span.ipc-rating-star--rating::text").get(), mais on prefere le data-testid plus robuste
            # pour info le.get(default='?') permet de retourner '?' si le sélecteur ne trouve rien, au lieu de retourner 'None'
            rating = film.css('span[data-testid="ratingGroup--imdb-rating"] ::text').get(default='?').strip()
            
            # ### On récupère le nombre de votants
            vote_parts = film.css("span.ipc-rating-star--voteCount::text").getall()
            nb_voteurs = parse_vote_count(vote_parts)
            
            # ### On a tout scrappé sur la 1ere page, on avance vers le lien detail du film
            # pour scraper le revenu brut mondial
            # detail_resp = requests.get(url_details, headers=headers, timeout=10)
            # detail_sel = Selector(text=detail_resp.text)    
            # revenu_brut_mondial = detail_sel.css( ... ).get(default='?').strip()
            # → Remplacé par :
           
            yield scrapy.Request(
                url=response.urljoin(film_relative_url),
                callback=self.parse_detail,
                meta={
                    'classement': classement,
                    'titre': titre,
                    'rating': rating,
                    'nb_voteurs': nb_voteurs,
                }
            )
            
            """ ce YIELD ci-dessus signifie : Quand tu auras téléchargé et parsé cette nouvelle page (la page détail du film),
            appelle automatiquement la méthode parse_detail et passe-lui la réponse. 
            
            Pour suivre les liens je pourrais aussi faire un 'response.follow' :
            yield response.follow(
                url=film_relative_url,              # relatif ou absolu
                callback=self.parse_detail,
                meta={...}
            )
            """


    # ici on estparti de la 1ere page et on est dans
    # la page détail du film, on peut scraper le revenu brut mondial
    def parse_detail(self, response):
        classement   = response.meta['classement']
        titre        = response.meta['titre']
        rating       = response.meta['rating']
        nb_voteurs   = response.meta['nb_voteurs']

        # Revenu brut mondial – plusieurs variantes pour plus de robustesse
        """
            Explication du sélecteur multiple pour revenu_brut_mondial :
            On utilise une union de plusieurs sélecteurs (séparés par des virgules)
            pour augmenter la robustesse face aux changements de structure ou de langue sur IMDb.

            Partie du sélecteur                                      | Ce qu'elle cible                                              | Quand elle est utile
            ----------------------------------------------------------|---------------------------------------------------------------|------------------------------------------------------
            li[data-testid="title-boxoffice-cumulativeworldwidegross"]| L'élément avec l'attribut data-testid spécifique              | Version la plus fiable quand IMDb utilise cet attribut
            span.ipc-metadata-list-item__list-content-item::text      | Toute span de valeur dans la liste des métadonnées            | Cas où le data-testid est absent ou changé, mais la classe de contenu reste
            li:contains("Gross worldwide")::text                      | N'importe quel <li> contenant exactement ce texte anglais     | Quand le site est en anglais et pas de data-testid
            li:contains("Montant brut mondial")::text                 | N'importe quel <li> contenant ce texte français               | Quand la page est en français
            li:contains("Cumulative Worldwide Gross")::text           | Variante anglaise plus longue (parfois utilisée)              | Anciennes versions ou variantes de la page

            Plus on ajoute de variantes, moins on risque de tout perdre si IMDb change une formulation.
        """
        revenu_brut_mondial = response.css(
            'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
            'span.ipc-metadata-list-item__list-content-item::text, '
            'li:contains("Gross worldwide")::text, '
            'li:contains("Montant brut mondial")::text, '
            'li:contains("Cumulative Worldwide Gross")::text'
        ).get(default='?').strip()

        # Nettoyage léger
        if revenu_brut_mondial != '?':
            revenu_brut_mondial = revenu_brut_mondial.replace('\xa0', ' ').strip()

        yield {
            'classement': classement,
            'title': titre,
            'url_details': response.url,
            'revenu_brut_mondial': revenu_brut_mondial,
            'rating': rating,
            'nb_voters': nb_voteurs
        }

#### Pour info la classe générée automatiquement par Scrapy (au bon endroit) pour ce projet ressemble à ça avant modification :


In [ ]:
import scrapy


class Imdb2Spider(scrapy.Spider):
    name = "imdb2"
    allowed_domains = ["imdb.com"]
    start_urls = ["https://imdb.com"]

    def parse(self, response):
        pass

#### 5. Exemple pour l'exercice : Create a new script called `imdb2.py` where you'll scrape the same information for all the movies in the chart!

Se placer dans le dossier racine du projet (le 1er imdb_scraper/) pour lancer les commandes Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    
└── imdb_scraper/                 
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```
ex:
```bash
cd semaine2\scrapy\movie_director_exercice\imdb_scraper
```
```bash
scrapy genspider imdb2 imdb.com
```
la sortie est celle-ci :
```
Created spider 'imdb2' using template 'basic' in module:
  imdb_scraper.spiders.imdb2
```
#### cela a généré le spider (imdb2.py) dans le dossier **spiders**.
-   Modifier / compléter ce fichier qui contient une classe **Imdb1Spider** qui hérite de **scrapy.Spider**
-   Vérifier avec :
```bash
scrapy list
imdb1
imdb2


## Part 2 - Optional 💪💪
6. Based on the results obtained, create a list called `url_list` containing all the urls for the movies in the charts.

2023-09-27 17:04:16 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


,ranking,title,url,total_earnings,rating,nb_voters
0,1,The Nun II,/title/tt10160976/?ref_=chtbo_t_1,$70M,5.9,19K
1,2,Expend4bles,/title/tt3291150/?ref_=chtbo_t_2,$8.7M,5.2,5K
2,3,A Haunting in Venice,/title/tt22687790/?ref_=chtbo_t_3,$26M,6.8,20K
3,4,The Equalizer 3,/title/tt17024450/?ref_=chtbo_t_4,$82M,7.1,21K
4,5,Barbie,/title/tt1517268/?ref_=chtbo_t_5,$631M,7.1,334K


url list: ['https://www.imdb.com/title/tt10160976/?ref_=chtbo_t_1', 'https://www.imdb.com/title/tt3291150/?ref_=chtbo_t_2', 'https://www.imdb.com/title/tt22687790/?ref_=chtbo_t_3']


7. Store the list of urls to a file called `url_list.txt`

8. Scrape all the pages from the movies in the charts and extract the complete cast, the Storyline, and the genres, also save the title and url for future joins purposes. *Pay attention to inconsistencies between pages*

2023-09-27 17:04:47 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-09-27 17:04:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.16 (main, Jan 11 2023, 16:05:54) - [GCC 11.2.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1t  7 Feb 2023), cryptography 38.0.4, Platform Linux-5.10.102.1-microsoft-standard-WSL2-x86_64-with-glibc2.31
2023-09-27 17:04:47 [scrapy.addons] INFO: Enabled addons:
[]
2023-09-27 17:04:47 [py.warnings] WARNING: /home/antoineco/miniconda3/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See th